# 🇲🇬 Voambolana Malagasy — NLP Project

Ce notebook centralise la collecte de données et la préparation du modèle IA.

### 📑 Étapes d'utilisation :
1.  **Exécutez la cellule de configuration** ci-dessous pour connecter votre Drive et récupérer le code.
2.  **Lancez les scrapers** (Phases 1-4) pour collecter les paroles.
3.  **Nettoyez et Consolidez** (Phases 5-6) : Fusion Chansons + Bible + Wikipédia.
4.  **Fondations NLP** (Phase 7-10) : Tokenizer, Normalisation et Embeddings.
5.  **Applications** (Phase 12) : Testez le moteur de recherche et le correcteur.

## 🛠️ 1. Configuration (Drive + Git)

In [ ]:
from google.colab import drive
import os

# 1. Monter le Drive
print("🔗 Connexion à Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Drive monté avec succès !")
except Exception as e:
    print("❌ Erreur lors du montage du Drive.")

# 2. Se placer dans le dossier du projet
REPO_NAME = "nlp_malagasy"
BASE_PATH = "/content/drive/MyDrive/" + REPO_NAME

if not os.path.exists(BASE_PATH):
    %cd /content/drive/MyDrive
    !git clone https://github.com/franck504/nlp_malagasy.git

%cd $BASE_PATH
!git pull origin main

print(f"\n📍 Position actuelle : {os.getcwd()}")
!ls

# 🏗️ PARTIE 1 : COLLECTE DES DONNÉES (Scraping)

### 🎵 Phase 1-3 : Discovery, Scraping & Stats

In [ ]:
!pip install requests beautifulsoup4
%cd $BASE_PATH/tononkira_rehetra

print("--- PHASE 1 : DISCOVERY ---")
!python 01_discover_artists.py --delay 1.0

print("\n--- PHASE 2 : SCRAPING (TURBO) ---")
!python 02_scrape_lyrics.py --delay 0.5 --artist-workers 4 --song-workers 10

print("\n--- PHASE 3 : STATISTIQUES ---")
!python 03_stats.py

### 📂 Phase 4 : Fusion du Corpus Brut

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 04_merge_corpus.py

# 🧹 PARTIE 2 : NETTOYAGE ET CONSOLIDATION

### 🧼 Phase 5 : Purification (Nettoyage de Langue)

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 05_clean_corpus.py --input malagasy_lyrics_corpus.txt --output malagasy_lyrics_cleaned.txt

### 🌍 Phase 6 : Wikipédia Malagasy (Extraction)

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 10_download_wikipedia.py
!python 11_extract_wiki.py

### 📖 Phase 4.5 : Consolidation FINALE (Lyrics + Bible + Wikipedia)

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 06_consolidate_corpus.py --lyrics malagasy_lyrics_cleaned.txt --bible ../from_bible_json --wiki ../malagasy_wikipedia_raw.txt --output malagasy_corpus_v2_final.txt

# 🧠 PARTIE 3 : FONDATIONS NLP

In [ ]:
!pip install tokenizers gensim
%cd $BASE_PATH/tononkira_rehetra

print("--- TOKENIZATION ---")
!python 07_train_tokenizer.py --corpus malagasy_corpus_v2_final.txt --output tokenizer_mg

print("\n--- NORMALIZATION ---")
!python 09_normalize_corpus.py --input malagasy_corpus_v2_final.txt --output malagasy_corpus_normalized.txt

print("\n--- EMBEDDINGS (FASTTEXT) ---")
!python 08_train_embeddings.py --corpus malagasy_corpus_normalized.txt --output embeddings_mg

# 🚀 PARTIE 4 : APPLICATIONS (DÉMO)

### 🏗️ 1. Indexation Sémantique
Analyse le dossier `/output` pour permettre la recherche par sens.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 12_test_app.py --index

### 🔍 2. Tableau de Bord de Test
Modifiez les paramètres ci-dessous pour tester l'IA.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra

RECHERCHE = "Fitiavana sy fahatokiana"  # @param {type:"string"}
CORRECTION = "fityavana"               # @param {type:"string"}
CONCEPT = "Antananarivo"               # @param {type:"string"}

print(f"--- RECHERCHE SÉMANTIQUE : {RECHERCHE} ---")
!python 12_test_app.py --query "{RECHERCHE}"

print(f"\n--- CORRECTION ORTHOGRAPHIQUE : {CORRECTION} ---")
!python 12_test_app.py --spell "{CORRECTION}"

print(f"\n--- EXPLORATION DE CONCEPT : {CONCEPT} ---")
!python 12_test_app.py --explore "{CONCEPT}"